In [1]:
import cv2

# === Load DNN Models ===
face_detector = cv2.FaceDetectorYN.create(
    "face_detection_yunet_2023mar.onnx", "", (320, 320),
    score_threshold=0.9, nms_threshold=0.3, top_k=5000
)
face_recognizer = cv2.FaceRecognizerSF.create("face_recognition_sface_2021dec.onnx", "")

# === Load Known Face and Extract Features ===
known_img = cv2.imread(r"C:\Users\veyadav\Documents\OneDrive - Capgemini\Documents\INTUDO\python\notebook\faces\dan\dan.jpg")
face_detector.setInputSize((known_img.shape[1], known_img.shape[0]))
_, known_faces = face_detector.detect(known_img)

# Skip if no face is detected
if known_faces is None or len(known_faces) == 0:
    raise ValueError("No face found in known_face.jpg")

aligned_known = face_recognizer.alignCrop(known_img, known_faces[0])
known_feature = face_recognizer.feature(aligned_known)

# === Open Video ===
cap = cv2.VideoCapture(r'C:\Users\veyadav\Downloads\Drupal-Branching.mp4' )

# === Target screen size ===
screen_width, screen_height = 1280, 720

cv2.namedWindow("DNN Face Recognition", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("DNN Face Recognition", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    face_detector.setInputSize((frame.shape[1], frame.shape[0]))
    _, faces = face_detector.detect(frame)

    if faces is not None:
        for i in range(faces.shape[0]):
            box = faces[i, :4].astype(int)

            aligned_face = face_recognizer.alignCrop(frame, faces[i])
            feature = face_recognizer.feature(aligned_face)

            score = face_recognizer.match(known_feature, feature, cv2.FaceRecognizerSF_FR_COSINE)
            label = "Match-Found" if score >= 0.363 else "Not Found"

            x, y, w, h = box
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({score:.2f})", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # === Resize frame to fit screen ===
    fh, fw = frame.shape[:2]
    scale = min(screen_width / fw, screen_height / fh)
    new_w, new_h = int(fw * scale), int(fh * scale)
    resized_frame = cv2.resize(frame, (new_w, new_h))

    canvas = cv2.copyMakeBorder(
        resized_frame,
        top=(screen_height - new_h) // 2,
        bottom=(screen_height - new_h + 1) // 2,
        left=(screen_width - new_w) // 2,
        right=(screen_width - new_w + 1) // 2,
        borderType=cv2.BORDER_CONSTANT,
        value=[0, 0, 0]
    )

    cv2.imshow("DNN Face Recognition", canvas)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
